In [1]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [ ]:
avisos = pd.read_csv('Predecir/avisos_detalles_c.csv')
postulantes = pd.read_csv('Predecir/postulantes_datos_c.csv')
postulaciones = pd.read_csv('Predecir/df_postulaciones_con_0.csv')
to_predict = pd.read_csv('Predecir/test_final_100k.csv')

In [3]:
postulaciones = postulaciones.sample(frac = 0.1)

In [4]:
del postulantes['Unnamed: 0']
del avisos['Unnamed: 0']
del postulaciones['Unnamed: 0']

In [5]:
avisos_si = avisos.copy()
#avisos_si['se_postulo'] = 1

In [6]:
postulaciones.shape

(847387, 3)

Muy grande para procesar, procesamos por bloques random y calculamos un promedio general

In [7]:
#postulaciones = postulaciones.sample(frac=0.15)

In [8]:
postulaciones.shape

(847387, 3)

In [9]:
postulaciones = pd.merge(postulaciones, avisos_si, how='inner', on=['idaviso'])

In [10]:
postulaciones.shape

(847543, 38)

In [11]:
postulaciones = pd.merge(postulaciones, postulantes, how='inner', on=['idpostulante'])

In [12]:
postulaciones.shape

(866757, 45)

Contamos cuantos postulados = 1 tenemos para ver cuantos 0 ponemos

In [13]:
postulaciones.shape

(866757, 45)

Probamos un 50% aprox de ceros en relacion a los unos

In [14]:
postulaciones.head()
#len(postulaciones[postulaciones["se_postulo"]==1])

,idaviso,idpostulante,se_postulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,capacitacion,junior,disponibilidad_viajar,...,fecha_inicial_a,fecha_final_a,cant_vistas,nombre,estado,sexo,edad,fecha_inicial_p,fecha_final_p,cantidad_vistas
0,1112378612,VNeJEqm,1.0,1,4,46,1864,0,0,0,...,5,24,4437,0,2,1,22.0,18,26,67
1,1112401940,VNeJEqm,1.0,1,4,1,3376,0,0,0,...,14,25,2595,0,2,1,22.0,18,26,67
2,1112399923,VNeJEqm,0.0,1,1,29,44,0,0,0,...,14,13,463,0,2,1,22.0,18,26,67
3,1112378612,8KorYj,1.0,1,4,46,1864,0,0,0,...,5,24,4437,3,2,1,29.0,7,12,1351
4,1112389521,8KorYj,1.0,1,2,10,490,0,0,0,...,9,30,4434,3,2,1,29.0,7,12,1351


In [15]:
def random_fill(row):
    return np.random.choice(avisos['idaviso'])

In [16]:
temp = len(postulaciones[postulaciones["se_postulo"]==1])
while (temp * 0.11> len(postulaciones[postulaciones["se_postulo"]==0])):
    #relleno = postulantes.sample(frac=0.5).copy()
    relleno = postulantes.copy()
    relleno['idaviso'] = np.nan
    relleno['idaviso'] = relleno.apply(random_fill, axis=1)
    relleno = pd.merge(relleno, avisos, how='inner', on=['idaviso'])
    postulaciones = pd.merge(postulaciones, relleno, how='outer')
    postulaciones = postulaciones.fillna({'se_postulo' : 0})
    print(len(postulaciones[postulaciones["se_postulo"]==0]))

In [17]:
postulaciones.shape

(866757, 45)

In [18]:
se_postulo = postulaciones['se_postulo']
del postulaciones['se_postulo']
del postulaciones['idaviso']
del postulaciones['idpostulante']

# DecisionTree

In [19]:
decisiontree = DecisionTreeClassifier(random_state=103040)
decisiontree.get_params()

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 103040,
 'splitter': 'best'}

In [20]:
X_train, X_test, y_train, y_test = train_test_split(postulaciones, se_postulo, test_size=0.30, random_state=42)

In [21]:
param_grid= {'max_features': [5,6,7,8,9,10,11,12,13,14]}
grid_drop = GridSearchCV(decisiontree, param_grid, cv=10)
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'max_features': 14} 0.7670442652320888


In [22]:
grid_drop.best_estimator_.feature_importances_  

array([0.00594282, 0.02407826, 0.03833163, 0.06096669, 0.00523711,
       0.00271822, 0.00349945, 0.00306131, 0.00629103, 0.00199125,
       0.00587751, 0.00349762, 0.00490737, 0.00615321, 0.0063994 ,
       0.0024741 , 0.00625052, 0.00413941, 0.00781078, 0.00483898,
       0.00688639, 0.00455476, 0.00670245, 0.00212417, 0.00450389,
       0.00522418, 0.00248446, 0.00467384, 0.00410301, 0.00556695,
       0.00208807, 0.03708274, 0.05732601, 0.04735043, 0.06152429,
       0.04152312, 0.02646876, 0.01418617, 0.11150884, 0.0964734 ,
       0.09640401, 0.1567734 ])

In [23]:
postulaciones.columns

Index(['tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa', 'capacitacion', 'junior',
       'disponibilidad_viajar', 'supervisor', 'marketing', 'terciario',
       'ingles', 'incorp_inmediata', 'multinacional', 'estudiantes',
       'proactivo', 'cajero', 'experiencia', 'vendedor', 'conoc_informatica',
       'relaciones_pers', 'administrativo', 'crecimiento', 'desarrollador',
       'jefe', 'analista', 'disponibilidad_horaria', 'flexibilidad',
       'clima laboral', 'ingeniero', 'secundario', 'web', 'online',
       'fecha_inicial_a', 'fecha_final_a', 'cant_vistas', 'nombre', 'estado',
       'sexo', 'edad', 'fecha_inicial_p', 'fecha_final_p', 'cantidad_vistas'],
      dtype='object')

In [24]:
result = grid_drop.best_estimator_.predict(X_test)

In [25]:
df = pd.DataFrame({'se_postulo':result})

In [26]:
df["se_postulo"].value_counts()

1.0    199436
0.0     60592
Name: se_postulo, dtype: int64

In [27]:
roc_auc_score(y_test, result)

0.6800994061871961

In [28]:
decisiontree = DecisionTreeRegressor(random_state=103040)
decisiontree.get_params()

{'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 103040,
 'splitter': 'best'}

In [29]:
X_train, X_test, y_train, y_test = train_test_split(postulaciones, se_postulo, test_size=0.30, random_state=42)

In [30]:
param_grid= {'max_features': [9,8,7,11, 10,12]}
grid_drop = GridSearchCV(decisiontree, param_grid, cv=10)
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'max_features': 12} -0.326617983892982


In [31]:
grid_drop.best_estimator_.feature_importances_  

array([0.0066531 , 0.02131915, 0.03677669, 0.06051365, 0.0055474 ,
       0.00235237, 0.00324098, 0.00318529, 0.00651387, 0.00145221,
       0.00492384, 0.00346339, 0.0044743 , 0.0070375 , 0.00644369,
       0.00261031, 0.00655549, 0.00385508, 0.00790736, 0.00576443,
       0.00695512, 0.00477468, 0.0065173 , 0.00204606, 0.00417541,
       0.00577969, 0.00293004, 0.00466246, 0.00391207, 0.00628243,
       0.00261385, 0.03178964, 0.06325244, 0.0440076 , 0.05731721,
       0.04229551, 0.0282262 , 0.0164151 , 0.11581336, 0.09793634,
       0.09863499, 0.15307243])

In [32]:
result = grid_drop.best_estimator_.predict(X_test)

In [33]:
df = pd.DataFrame({'se_postulo':result})

In [34]:
df["se_postulo"].value_counts()

1.0    199700
0.0     60328
Name: se_postulo, dtype: int64

In [35]:
roc_auc_score(y_test, result)

0.6790635349160524

In [88]:
to_predict = pd.read_csv('Predecir/test_final_100k.csv')

In [89]:

to_predict = pd.merge(to_predict, avisos, on = "idaviso",how="left")
to_predict = pd.merge(to_predict, postulantes, on = "idpostulante",how="left")
to_predict = to_predict.drop_duplicates(["idaviso","idpostulante"])

In [90]:
to_predict.shape

(100000, 45)

0.0    93150
1.0     4144
Name: web, dtype: int64

In [91]:

del to_predict["idaviso"]
del to_predict["idpostulante"]

In [92]:
to_predict = to_predict.dropna()

In [94]:
to_sumit = grid_drop.best_estimator_.predict(to_predict.drop("id",axis = "columns"))

In [95]:
to_predict["prediccion"] = to_sumit

In [96]:
res = pd.read_csv('Predecir/test_final_100k.csv')

res = pd.merge(res, avisos, on = "idaviso",how="left")
res = pd.merge(res, postulantes, on = "idpostulante",how="left")
res = res.drop_duplicates(["idaviso","idpostulante"])

In [108]:
to_sumit = pd.merge(res,to_predict,on="id",how="left")["prediccion"]
to_sumit = to_sumit.fillna(0.5)
df = pd.DataFrame()
df["sepostulo"] = to_sumit
df["id"] = pd.read_csv('Predecir/test_final_100k.csv')["id"]
final = df.set_index("id")
final.shape

(100000, 1)

In [109]:
final.to_csv("final.csv")